<a href="https://colab.research.google.com/github/s14t284/Seq2Seq_sample/blob/master/Seq2Seq_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seq2Seq Sample

Seq2Seqのサンプル実装

- deep learning ライブラリでseq2seqを組むとき，どういう風に実装するかのサンプル
- 作成した動機
    - どのくらい，ライブラリはNNを抽象化しているかを知ってほしいため作成
    - ライブラリの使い方のサンプル（chainer以外の日本語サンプルはかなり少ない）
- 今回は，流行しているDLライブラリの1つであるpytorchで作成

## 必要なもの
- google colaboratory(導入は[ここ](https://tutorials.chainer.org/ja/01_Welcome_to_Chainer_Tutorial.html)がわかりやすい) or GPUが導入されたマシン
- データセット(今回は[ここ](https://nlp.stanford.edu/projects/jesc/)からダウンロード．日本語と英語の字幕の対訳コーパス)

In [48]:
!pip install torchtext==0.4.0
!pip install mecab-python3

     |████████████████████████████████| 61kB 7.5MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## DataLoader
- データセットをよしなに扱ってくれるクラス
- 今回はpytorchで実装するが，tensorflowでもDataLoaderに似た概念は存在する
- pytorchでは，実装済みのDataLoaderが外部ライブラリで用意されている．今回は`torchtext`を利用する
    - 画像 : torchvision
    - 言語 : torchtext
    - DataLoaderを自力で1から書くのはつらみなので，出来るだけ既存のものに頼る．
        - ただし，torchtextはドキュメントが...

In [0]:
import MeCab
import torchtext
import torch
import torch.nn as nn
import spacy
from torchtext import data
from torchtext import datasets
from torchtext.vocab import FastText
from torchtext.vocab import GloVe

In [0]:
m = MeCab.Tagger('-Owakati')
def jap_tokenizer(text):
    text = text.replace(' ', '')
    return [tok for tok in m.parse(text).split(' ')][:-1]

spacy_en = spacy.load('en')
def eng_tokenizer(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [35]:
vars(a[2])

{'ENG': ['be', 'careful', '.'], 'JAP': ['注意', 'しろ', '\n']}

### データの読み込み
- とても時間がかかるので注意

In [0]:
data_path =  '/content/drive/My Drive/'
english = data.Field(sequential=True, tokenize=eng_tokenizer, lower=True)
japanese = data.Field(sequential=True, tokenize=jap_tokenizer)
train, dev, test = data.TabularDataset.splits(
    path=data_path + 'mt_dataset/', train='train', validation='dev', test='test', format='tsv',
    fields=[('ENG', english), ('JAP', japanese)]
)

In [46]:
english.build_vocab(train, test, min_freq=2, vectors=GloVe())
japanese.build_vocab(train, test, min_freq=2, vectors=FastText(language='ja'))


.vector_cache/wiki.ja.vec: 0.00B [00:00, ?B/s]


HTTPError: ignored

In [47]:
FastText(language='ja')


.vector_cache/wiki.ja.vec: 0.00B [00:00, ?B/s]


HTTPError: ignored

In [0]:
train_iter = data.BucketIterator(train, batch_size=64,
                                                        sort_key=lambda x: data.interleave_keys(len(x.english), len(x.japanese)))

In [0]:
class Encoder(nn.Module):
    
    def __init__(self, embedding_dim: int,
                           hidden_dim: int, batch_size: int):
        super(Encoder, self).__init__()
        self._hidden_dim = hidden_dim
        self._batch_size = batch_size
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
    
    def forward(self, x):
        """
        pytorchではnn.Moduleのforwardメソッドに順伝播を実装する
        """
        
        # Noneを隠れ層の値に当たる引数に挿入することで，0で初期化されたベクトルを隠れ層として挿入
        _, (h, c) = self.lstm(x, None)
        return h
    
class Decoder(nn.Module):
    
    def __init__(self, vocab_size: int, embedding_dim: int,
                           hidden_dim: int, batch_size: int):
        super(Encoder, self).__init__()
        self._hidden_dim = hidden_dim
        self._batch_size = batch_size
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden, cell):
        y, (hidden, cell) = self.lstm(x, (hidden, cell))
        out = self.linear(y)
        return out, hidden, cell

In [0]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        
        